In [3]:
# ResNet
 
import torch
from torch import nn, optim
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F
 
batch_size = 64
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081))])
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transforms)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
 
test_dataset = datasets.MNIST(root='../dataset/mnist', train=True, download=True, transform=transforms)
test_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
 

100%|██████████| 9912422/9912422 [00:00<00:00, 120244421.69it/s]


Extracting ../dataset/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 93181302.94it/s]


Extracting ../dataset/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 36930843.81it/s]


Extracting ../dataset/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22412386.79it/s]


Extracting ../dataset/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw



In [4]:
 
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.channels = channels
 
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
 
    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        return F.relu(x + y)
 
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
 
        self.rbloch1 = ResidualBlock(16)
        self.rbloch2 = ResidualBlock(32)
 
        self.fc = nn.Linear(512, 10)
 
    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.rbloch1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.rbloch2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x
 
 

In [5]:
model = Net()
 
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
 

In [6]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
 
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_size + 1, running_loss / 300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
 
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
 
        print('accuracy on test set: %d %% ' % (100 * correct / total))
 
 

In [7]:
for epoch in range(10):
    train(epoch)
    test()
 

[1,    65] loss: 0.525
[1,    65] loss: 0.168
[1,    65] loss: 0.111
accuracy on test set: 96 % 
[2,    65] loss: 0.092
[2,    65] loss: 0.079
[2,    65] loss: 0.076
accuracy on test set: 97 % 
[3,    65] loss: 0.060
[3,    65] loss: 0.058
[3,    65] loss: 0.059
accuracy on test set: 98 % 
[4,    65] loss: 0.046
[4,    65] loss: 0.050
[4,    65] loss: 0.046
accuracy on test set: 98 % 
[5,    65] loss: 0.042
[5,    65] loss: 0.039
[5,    65] loss: 0.039
accuracy on test set: 98 % 
[6,    65] loss: 0.036
[6,    65] loss: 0.036
[6,    65] loss: 0.034
accuracy on test set: 99 % 
[7,    65] loss: 0.033
[7,    65] loss: 0.029
[7,    65] loss: 0.030
accuracy on test set: 99 % 
[8,    65] loss: 0.024
[8,    65] loss: 0.031
[8,    65] loss: 0.025
accuracy on test set: 99 % 
[9,    65] loss: 0.021
[9,    65] loss: 0.027
[9,    65] loss: 0.026
accuracy on test set: 99 % 
[10,    65] loss: 0.022
[10,    65] loss: 0.023
[10,    65] loss: 0.022
accuracy on test set: 99 % 
